# feature_generation

In [1]:
import pandas as pd
from datetime import datetime

from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

In [2]:
import nbimporter
from pre_processing import load_train, load_test

Importing Jupyter notebook from pre_processing.ipynb


### Funciones auxiliares

In [3]:
def corr(df):
    return df.corr()['precio'].sort_values(ascending=False)[1:]

In [4]:
def aniomes(anio, mes):
    if len(str(mes)) == 1:
        return int(str(anio)+'0'+str(mes))
    return int(str(anio)+str(mes))

In [5]:
def contiene_indicadores(descripcion, indicadores):
    for indicador in indicadores:
        if indicador in str(descripcion).lower():
            return 1
    return 0

In [6]:
def binning_habitaciones(x):
    if (x==-1):
        return 'unknown'
    elif x==1:
        return 'monoambiente'
    elif (x==2):
        return 'dos ambientes'
    elif (x==3):
        return 'tres ambientes'
    elif (x==4):
        return 'cuatro ambientes'
    elif (x==5):
        return 'cinco ambientes'
    else:
        return 'mas de 5 ambientes'

In [7]:
def binning_antiguedad(x):
    if (x==-1):
        return 'unknown'
    elif x==0:
        return 'new'
    elif (x<=5):
        return '0 a 5'
    elif (x<=10):
        return '5 a 10'
    elif (x<=15):
        return '10 a 15'
    elif (x<=20):
        return '15 a 20'
    elif (x<=30):
        return '20 a 30'
    elif (x<=50):
        return '30 a 50'
    else:
        return '+50'
    

# Desarollo

In [8]:
df = load_train()
df2 = load_test()

In [9]:
display(df.head(2))
print(df.shape)

,titulo,descripcion,direccion,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
id,,,,,,,,,,,,,,,,,,,,
254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Avenida Division del Norte 2005,Apartamento,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,2015-08-23,0,0,0,0,0,2273000.0
53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",AV. MEXICO,Casa en condominio,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,2013-06-28,0,0,0,1,1,3600000.0


(240000, 20)


In [10]:
display(df2.head(2))
print(df2.shape)

,titulo,descripcion,direccion,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
id,,,,,,,,,,,,,,,,,,,
4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Bosque de Cedros,Casa,Miguel Hidalgo,Distrito Federal,29.0,3.0,NaN,4.0,300.0,NaN,NaN,2013-07-20,0,0,0,0,0
51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,NaN,Apartamento,Mérida,Yucatán,NaN,1.0,1.0,1.0,67.0,67.0,113851.0,2015-10-24,0,0,0,0,0


(60000, 19)


In [11]:
features = {}
features['originales'] = df.columns.values

### Variables categoricas (encoding)

In [12]:
var_categoricas = ['tipodepropiedad', 'ciudad', 'provincia']

In [13]:
# label encoding
encoder = LabelEncoder()
encoded = df[var_categoricas].apply(encoder.fit_transform).add_suffix("_labelencoded")
encoded.to_csv('./features/train/categoricas_labelencoded.csv')
encoder = LabelEncoder()
encoded = df2[var_categoricas].apply(encoder.fit_transform).add_suffix("_labelencoded")
encoded.to_csv('./features/test/categoricas_labelencoded.csv')
features['categoricas_labelencoded'] = encoded.columns.values

In [14]:
# onehot 

# para tipodepropiedad
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df['tipodepropiedad']).add_suffix("_ohencoded")
encoded.to_csv('./features/train/tipodepropiedad_ohencoded.csv')
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df2['tipodepropiedad']).add_suffix("_ohencoded")
encoded.to_csv('./features/test/tipodepropiedad_ohencoded.csv')
features['tipodepropiedad_ohencoded'] = encoded.columns.values

# para provincias
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df['provincia']).add_suffix("_ohencoded")
encoded.to_csv('./features/train/provincia_ohencoded.csv')
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df2['provincia']).add_suffix("_ohencoded")
encoded.to_csv('./features/test/provincia_ohencoded.csv')
features['provincia_ohencoded'] = encoded.columns.values

# para ciudades
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df['ciudad']).add_suffix("_ohencoded")
encoded.to_csv('./features/train/ciudad_ohencoded.csv')
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df2['ciudad']).add_suffix("_ohencoded")
encoded.to_csv('./features/test/ciudad_ohencoded.csv')
features['ciudad_ohencoded'] = encoded.columns.values


### Generacion de nuevas features

In [15]:
df.columns

Index(['titulo', 'descripcion', 'direccion', 'tipodepropiedad', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'fecha', 'gimnasio',
       'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

##### FECHA

In [16]:
df_fecha = df[['fecha', 'precio', 'metroscubiertos']].copy()
df_fecha2 = df2[['fecha', 'metroscubiertos']].copy()

df_fecha['anio'] = df_fecha['fecha'].dt.year
df_fecha['mes'] = df_fecha['fecha'].dt.month
df_fecha['dia'] = df_fecha['fecha'].dt.day
df_fecha['timestamp'] = df_fecha['fecha'].apply(lambda x: datetime.timestamp(x))
df_fecha['aniomes'] = df_fecha.apply(lambda row: aniomes(row['anio'], row['mes']), axis=1)

df_fecha2['anio'] = df_fecha2['fecha'].dt.year
df_fecha2['mes'] = df_fecha2['fecha'].dt.month
df_fecha2['dia'] = df_fecha2['fecha'].dt.day
df_fecha2['timestamp'] = df_fecha2['fecha'].apply(lambda x: datetime.timestamp(x))
df_fecha2['aniomes'] = df_fecha2.apply(lambda row: aniomes(row['anio'], row['mes']), axis=1)

In [17]:
# calculamos el precio promedio del metro cubierto por aniomes
precio_mtcubierto_aniomes = df_fecha.groupby(['aniomes'])['metroscubiertos', 'precio'].agg(sum).apply(lambda x: x['precio']/x['metroscubiertos'], axis=1)

df_fecha['precio_promedio_metrocubierto_aniomes'] = df_fecha['aniomes'].apply(lambda x: precio_mtcubierto_aniomes[x])
df_fecha2['precio_promedio_metrocubierto_aniomes'] = df_fecha2['aniomes'].apply(lambda x: precio_mtcubierto_aniomes[x])

In [18]:
# calculamos el precio promedio del metro cubierto por mes
precio_mtcubierto_mes = df_fecha.groupby(['mes'])['metroscubiertos', 'precio'].agg(sum).apply(lambda x: x['precio']/x['metroscubiertos'], axis=1)

df_fecha['precio_promedio_metrocubierto_mes'] = df_fecha['mes'].apply(lambda x: precio_mtcubierto_mes[x])
df_fecha2['precio_promedio_metrocubierto_mes'] = df_fecha2['mes'].apply(lambda x: precio_mtcubierto_mes[x])

In [19]:
df_fecha = df_fecha.drop(['precio', 'metroscubiertos'], axis=1)
df_fecha2 = df_fecha2.drop(['metroscubiertos'], axis=1)

pasar_a_enteros = ['precio_promedio_metrocubierto_mes', 'precio_promedio_metrocubierto_aniomes']

df_fecha[pasar_a_enteros] = df_fecha[pasar_a_enteros].astype('uint16')
df_fecha2[pasar_a_enteros] = df_fecha2[pasar_a_enteros].astype('uint16')

In [20]:
df_fecha.to_csv('./features/train/fecha.csv')
df_fecha2.to_csv('./features/test/fecha.csv')
features['fecha'] = df_fecha.columns.values

In [21]:
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_fecha['aniomes'].astype('category')).add_suffix("_ohencoded")
encoded.to_csv('./features/train/aniomes_ohencoded.csv')
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_fecha2['aniomes'].astype('category')).add_suffix("_ohencoded")
encoded.to_csv('./features/test/aniomes_ohencoded.csv')

features['aniomes_ohencoded'] = encoded.columns.values

##### DESCRIPCION

In [22]:
df.columns

Index(['titulo', 'descripcion', 'direccion', 'tipodepropiedad', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'fecha', 'gimnasio',
       'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio'],
      dtype='object')

In [23]:
df_descripcion = df[['descripcion']].copy()
df_descripcion2 = df2[['descripcion']].copy()

indicadores = ["jardin", "patio", "jardín"]
df_descripcion["jardin"] = df_descripcion["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))
df_descripcion2["jardin"] = df_descripcion2["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))

indicadores = ["vigilancia", "seguridad"]
df_descripcion["vigilancia"] = df_descripcion["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))
df_descripcion2["vigilancia"] = df_descripcion2["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))

indicadores = ["aire acondicionado", "aires acondicionados"]
df_descripcion["aire_acondicionado"] = df_descripcion["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))
df_descripcion2["aire_acondicionado"] = df_descripcion2["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))

indicadores = ["ventilador", "ventiladores"]
df_descripcion["ventilador"] = df_descripcion["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))
df_descripcion2["ventilador"] = df_descripcion2["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))

indicadores = ["calefaccion", "caloventor", "estufa", "calefacción", "calentador"]
df_descripcion["calefaccion"] = df_descripcion["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))
df_descripcion2["calefaccion"] = df_descripcion2["descripcion"].map(lambda x: contiene_indicadores(x, indicadores))

In [24]:
df_descripcion.drop('descripcion', inplace=True, axis=1)
df_descripcion2.drop('descripcion', inplace=True, axis=1)

df_descripcion.to_csv('./features/train/descripcion.csv')
df_descripcion2.to_csv('./features/test/descripcion.csv')
features['descripcion'] = df_descripcion.columns.values

##### MEAN Y STD DE SUPERFICIE CUBIERTA DE LA PROVINCIA

In [25]:
df_superficie_cubierta = df[['metroscubiertos','ciudad']].copy().reset_index()
df_superficie_cubierta2 = df2[['metroscubiertos','ciudad']].copy().reset_index()

avg_metroscubiertos = df_superficie_cubierta.groupby('ciudad').agg({"metroscubiertos":["mean","std"]}).reset_index()
avg_metroscubiertos2 = df_superficie_cubierta2.groupby('ciudad').agg({"metroscubiertos":["mean","std"]}).reset_index()
avg_metroscubiertos.columns = ['ciudad','metroscubiertos_mean','metroscubiertos_std']
avg_metroscubiertos2.columns = ['ciudad','metroscubiertos_mean','metroscubiertos_std']

df_superficie_cubierta_mean_std = pd.merge(df_superficie_cubierta,avg_metroscubiertos,how='inner',on='ciudad').set_index('id')
df_superficie_cubierta_mean_std2 = pd.merge(df_superficie_cubierta2,avg_metroscubiertos2,how='inner',on='ciudad').set_index('id')

df_superficie_cubierta_mean_std = df_superficie_cubierta_mean_std[["metroscubiertos_mean","metroscubiertos_std"]]
df_superficie_cubierta_mean_std2 = df_superficie_cubierta_mean_std2[["metroscubiertos_mean","metroscubiertos_std"]]

df_superficie_cubierta_mean_std.to_csv('./features/train/superficie_mean_std.csv')
df_superficie_cubierta_mean_std2.to_csv('./features/test/superficie_mean_std.csv')

##### BINNING (+OHE)

In [26]:
# binning para antiguedad

df_antiguedad = df[['antiguedad']].copy()
df_antiguedad2 = df2[['antiguedad']].copy()

In [27]:
df_antiguedad['antiguedad'].fillna(-1, inplace=True)
df_antiguedad2['antiguedad'].fillna(-1, inplace=True)

df_antiguedad['antiguedad'] = df_antiguedad['antiguedad'].astype(int)
df_antiguedad2['antiguedad'] = df_antiguedad2['antiguedad'].astype(int)

df_antiguedad['antiguedad_binning'] = df_antiguedad['antiguedad'].map(lambda x: binning_antiguedad(x))
df_antiguedad2['antiguedad_binning'] = df_antiguedad2['antiguedad'].map(lambda x: binning_antiguedad(x))

In [28]:
# ohe
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_antiguedad['antiguedad_binning']).add_suffix("_ohencoded")
encoded.to_csv('./features/train/antiguedad_binning.csv')
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_antiguedad2['antiguedad_binning']).add_suffix("_ohencoded")
encoded.to_csv('./features/test/antiguedad_binning.csv')
features['antiguedad_binning'] = encoded.columns.values

In [29]:
# binning para habitaciones
df_habitaciones = df[['habitaciones']].copy()
df_habitaciones2 = df2[['habitaciones']].copy()

In [30]:
df_habitaciones['habitaciones'].fillna(-1, inplace=True)
df_habitaciones2['habitaciones'].fillna(-1, inplace=True)

df_habitaciones['habitaciones'] = df_habitaciones['habitaciones'].astype(int)
df_habitaciones2['habitaciones'] = df_habitaciones2['habitaciones'].astype(int)

df_habitaciones['hab_binning'] = df_habitaciones['habitaciones'].map(lambda x: binning_habitaciones(x))
df_habitaciones2['hab_binning'] = df_habitaciones2['habitaciones'].map(lambda x: binning_habitaciones(x))

In [31]:
# ohe
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_habitaciones['hab_binning']).add_suffix("_ohencoded")
encoded.to_csv('./features/train/habitaciones_binning.csv')
encoder = ce.OneHotEncoder()
encoded = encoder.fit_transform(df_habitaciones2['hab_binning']).add_suffix("_ohencoded")
encoded.to_csv('./features/test/habitaciones_binning.csv')
features['habitaciones_binning'] = encoded.columns.values

# Full DF

In [32]:
def load_engineered_train():
    df = load_train()
    df = df[["centroscomercialescercanos","escuelascercanas","metroscubiertos","metrostotales","banos","antiguedad","habitaciones","garages","piscina","gimnasio","precio"]]
    
    #Agregado de anio-mes en one hot encoding
    df_aniomes_ohencoded = pd.read_csv("features/train/aniomes_ohencoded.csv", index_col='id')
    df = pd.merge(df,df_aniomes_ohencoded,how='left',left_index=True, right_index=True)
    
    #Agregado de provincia en one hot encoding
    df_provincia_ohencoded = pd.read_csv("features/train/provincia_ohencoded.csv", index_col='id')
    df = pd.merge(df,df_provincia_ohencoded,how='left',left_index=True, right_index=True)
    
    #Agregado de tipodepropiedad en one hot encoding
    df_tipo_ohencoded = pd.read_csv("features/train/tipodepropiedad_ohencoded.csv", index_col='id')
    df = pd.merge(df,df_tipo_ohencoded,how='left',left_index=True, right_index=True)
    
    #Agregado de ciudad en one hot encoding
    df_ciudad_ohencoded = pd.read_csv("features/train/ciudad_ohencoded.csv", index_col='id')
    df = pd.merge(df,df_ciudad_ohencoded,how='left',left_index=True, right_index=True)
    
    
    #Agregado de features obtenidos de la descripcion
    df_descripcion = pd.read_csv("features/train/descripcion.csv", index_col='id')
    df = pd.merge(df,df_descripcion,how='left',left_index=True, right_index=True)
    
        
    #Agregado de superficie mean y std por provincia
    df_datos_superficie = pd.read_csv("features/train/superficie_mean_std.csv", index_col='id')
    df = pd.merge(df,df_datos_superficie,how='left',left_index=True, right_index=True)

    
    df = df.reset_index()
    df.pop('id')
    
    return df

In [33]:
load_engineered_train()

,centroscomercialescercanos,escuelascercanas,metroscubiertos,metrostotales,banos,antiguedad,habitaciones,garages,piscina,gimnasio,...,ciudad_79_ohencoded,ciudad_80_ohencoded,ciudad_81_ohencoded,jardin,vigilancia,aire_acondicionado,ventilador,calefaccion,metroscubiertos_mean,metroscubiertos_std
0,0,0,80.0,80.0,2.0,NaN,2.0,1.0,0,0,...,0,0,0,0,0,0,0,0,126.665194,74.580317
1,1,1,268.0,180.0,2.0,10.0,3.0,2.0,0,0,...,0,0,0,0,0,0,0,0,235.751233,96.728660
2,0,0,144.0,166.0,2.0,5.0,3.0,2.0,0,0,...,0,0,0,1,1,0,0,0,105.858278,60.925944
3,1,1,63.0,67.0,1.0,1.0,2.0,1.0,0,0,...,0,0,0,0,1,0,0,0,158.017162,95.541916
4,0,0,95.0,95.0,1.0,10.0,2.0,1.0,0,0,...,0,0,1,0,0,0,0,0,182.015584,97.211357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,0,0,67.0,NaN,1.0,0.0,2.0,2.0,0,0,...,0,0,0,1,1,0,0,0,158.017162,95.541916
239996,1,1,200.0,250.0,3.0,0.0,3.0,3.0,0,0,...,0,0,0,1,0,0,0,0,143.767504,81.549224
239997,0,0,138.0,138.0,2.0,20.0,2.0,1.0,0,0,...,0,0,0,1,1,0,0,0,126.665194,74.580317
239998,1,1,235.0,137.0,4.0,20.0,4.0,0.0,0,1,...,0,0,0,0,0,0,0,0,132.732015,94.140068


# Resultados

In [88]:
features
# hardcodear diccionario para devolverlo

{'originales': array(['titulo', 'descripcion', 'direccion', 'tipodepropiedad', 'ciudad',
        'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
        'metroscubiertos', 'metrostotales', 'idzona', 'fecha', 'gimnasio',
        'usosmultiples', 'piscina', 'escuelascercanas',
        'centroscomercialescercanos', 'precio'], dtype=object),
 'categoricas_labelencoded': array(['tipodepropiedad_labelencoded', 'ciudad_labelencoded',
        'provincia_labelencoded'], dtype=object),
 'tipodepropiedad_ohencoded': array(['tipodepropiedad_1_ohencoded', 'tipodepropiedad_2_ohencoded',
        'tipodepropiedad_3_ohencoded', 'tipodepropiedad_4_ohencoded',
        'tipodepropiedad_5_ohencoded', 'tipodepropiedad_6_ohencoded',
        'tipodepropiedad_7_ohencoded', 'tipodepropiedad_8_ohencoded',
        'tipodepropiedad_9_ohencoded', 'tipodepropiedad_10_ohencoded'],
       dtype=object),
 'provincia_ohencoded': array(['provincia_1_ohencoded', 'provincia_2_ohencoded',
        'provincia_3